In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

#### Police Sentiment Data
Get average safety and trust scores for each district per month and year.

In [2]:
police_df_cleaned = pd.read_csv('/Users/Emi/Documents/GitHub/Crime-Analysis/police_df_cleaned.csv')
crime_df = pd.read_csv('Chicago_Crime_Data_Cleaned.csv')

In [3]:
crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats,Primary Type Grouping
0,5741943,HN549294,2007-08-25 09:22:18,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,2422,24.0,49.0,1.0,08A,NaN,NaN,2007,2015-08-17 15:03:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All Other Crime Types
1,1930689,HH109118,2002-01-05 21:24:00,007XX E 103 ST,0820,THEFT,$500 AND UNDER,GAS STATION,True,False,512,5.0,NaN,NaN,06,NaN,NaN,2002,2016-02-04 06:33:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All Other Crime Types
2,13203321,JG415333,2023-09-06 17:00:00,002XX N Wells st,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122,1.0,42.0,32.0,14,1174694.0,1901831.0,2023,2023-11-04 15:40:18,41.886018,-87.633938,"(41.886018055, -87.633937881)",22.0,14309.0,38.0,92.0,36.0,46.0,22.0,79.0,All Other Crime Types
3,13210088,JG423627,2023-08-31 12:00:00,023XX W JACKSON BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,STREET,False,False,1225,12.0,27.0,28.0,11,1160870.0,1898642.0,2023,2023-09-16 15:41:56,41.877565,-87.684791,"(41.877565108, -87.68479102)",48.0,21184.0,29.0,766.0,46.0,28.0,15.0,139.0,All Other Crime Types
4,13210004,JG422532,2023-07-24 21:45:00,073XX S JEFFERY BLVD,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,False,False,333,3.0,7.0,43.0,02,1190812.0,1856743.0,2023,2023-09-16 15:41:56,41.761919,-87.576209,"(41.7619185, -87.576209245)",32.0,22538.0,39.0,419.0,37.0,24.0,18.0,89.0,All Other Crime Types


In [4]:
#Keeping only overall safety and trust scores to start, we can add more later if necessary
police_df_filtered = police_df_cleaned[['DISTRICT','SAFETY','TRUST','START_DATE','END_DATE']]

In [5]:
crime_df.District = crime_df.District.astype('str')
len(list(crime_df.District.unique()))

25

In [6]:
police_df_filtered.DISTRICT = police_df_filtered.DISTRICT.astype('str')
len(list(police_df_filtered.DISTRICT.unique()))

/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/3451442942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_filtered.DISTRICT = police_df_filtered.DISTRICT.astype('str')


22

In [7]:
police_df_filtered['START_DATE'] = pd.to_datetime(police_df_filtered['START_DATE'])
police_df_filtered['END_DATE'] = pd.to_datetime(police_df_filtered['END_DATE'])

/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/2084439707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_filtered['START_DATE'] = pd.to_datetime(police_df_filtered['START_DATE'])
/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/2084439707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_filtered['END_DATE'] = pd.to_datetime(police_df_filtered['END_DATE'])


In [8]:
(police_df_filtered['END_DATE'] - police_df_filtered['START_DATE']).dt.days.value_counts()
#These all seem to show a time period of one month, so we can just create columns for the Month and Year

30    3731
29    2184
27     455
28      91
dtype: int64

In [9]:
police_df_filtered['Year_Month'] = police_df_filtered['START_DATE'].dt.to_period('M')

/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/1937159896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_df_filtered['Year_Month'] = police_df_filtered['START_DATE'].dt.to_period('M')


In [10]:
police_df_filtered.drop(columns=['START_DATE','END_DATE'])

,DISTRICT,SAFETY,TRUST,Year_Month
0,14.0,56.69,60.90,2021-04
1,5.0,35.51,47.52,2021-04
2,11.0,42.63,46.38,2021-04
3,9.0,46.02,63.57,2021-04
4,16.0,55.97,79.77,2021-04
...,...,...,...,...
6456,12.0,66.53,63.30,2023-09
6457,1.0,69.07,73.06,2023-09
6458,16.0,65.99,71.75,2023-09
6459,10.0,57.74,54.83,2023-09


In [11]:
police_df_filtered.head()

,DISTRICT,SAFETY,TRUST,START_DATE,END_DATE,Year_Month
0,14.0,56.69,60.90,2021-04-01,2021-04-30,2021-04
1,5.0,35.51,47.52,2021-04-01,2021-04-30,2021-04
2,11.0,42.63,46.38,2021-04-01,2021-04-30,2021-04
3,9.0,46.02,63.57,2021-04-01,2021-04-30,2021-04
4,16.0,55.97,79.77,2021-04-01,2021-04-30,2021-04


There are multiple safety and trust scores for each district since we removed sectors, so we can take the average scores for each district

In [12]:
mean_sentiment_scores = police_df_filtered.groupby(['DISTRICT','Year_Month'])['SAFETY','TRUST'].mean().round(2).reset_index()

/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/4264651156.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_sentiment_scores = police_df_filtered.groupby(['DISTRICT','Year_Month'])['SAFETY','TRUST'].mean().round(2).reset_index()


In [13]:
mean_sentiment_scores[mean_sentiment_scores.DISTRICT.eq('1.0')]

,DISTRICT,Year_Month,SAFETY,TRUST
0,1.0,2017-11,62.55,64.38
1,1.0,2017-12,63.03,63.73
2,1.0,2018-01,68.74,65.46
3,1.0,2018-02,73.82,66.11
4,1.0,2018-03,71.74,67.13
5,1.0,2018-04,70.83,66.21
6,1.0,2018-05,72.20,63.36
7,1.0,2018-06,70.54,63.33
8,1.0,2018-07,69.68,67.48
9,1.0,2018-08,69.63,69.75


In [14]:
mean_sentiment_scores.head()

,DISTRICT,Year_Month,SAFETY,TRUST
0,1.0,2017-11,62.55,64.38
1,1.0,2017-12,63.03,63.73
2,1.0,2018-01,68.74,65.46
3,1.0,2018-02,73.82,66.11
4,1.0,2018-03,71.74,67.13


In [15]:
crime_df['Date'] = pd.to_datetime(crime_df['Date'])

In [16]:
crime_df['Year_Month'] = crime_df['Date'].dt.to_period('M')

In [17]:
mean_sentiment_scores.rename(columns={'SAFETY':'Police Safety Score',
                                      'TRUST':'Police Trust Score',
                                      'DISTRICT':'Police Districts'},inplace=True)

In [18]:
crime_df['Police Districts'] = crime_df['Police Districts'].astype('str')

In [19]:
crime_and_police = pd.merge(crime_df,mean_sentiment_scores,how='left',on=['Police Districts','Year_Month'])
crime_and_police.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats,Primary Type Grouping,Year_Month,Police Safety Score,Police Trust Score
0,5741943,HN549294,2007-08-25 09:22:18,074XX N ROGERS AVE,0560,ASSAULT,SIMPLE,OTHER,False,False,2422,24.0,49.0,1.0,08A,NaN,NaN,2007,2015-08-17 15:03:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,All Other Crime Types,2007-08,NaN,NaN
1,1930689,HH109118,2002-01-05 21:24:00,007XX E 103 ST,0820,THEFT,$500 AND UNDER,GAS STATION,True,False,512,5.0,NaN,NaN,06,NaN,NaN,2002,2016-02-04 06:33:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,All Other Crime Types,2002-01,NaN,NaN
2,13203321,JG415333,2023-09-06 17:00:00,002XX N Wells st,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,122,1.0,42.0,32.0,14,1174694.0,1901831.0,2023,2023-11-04 15:40:18,41.886018,-87.633938,"(41.886018055, -87.633937881)",22.0,14309.0,38.0,92.0,36.0,46.0,22.0,79.0,All Other Crime Types,2023-09,62.32,63.51
3,13210088,JG423627,2023-08-31 12:00:00,023XX W JACKSON BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,STREET,False,False,1225,12.0,27.0,28.0,11,1160870.0,1898642.0,2023,2023-09-16 15:41:56,41.877565,-87.684791,"(41.877565108, -87.68479102)",48.0,21184.0,29.0,766.0,46.0,28.0,15.0,139.0,All Other Crime Types,2023-08,54.24,54.21
4,13210004,JG422532,2023-07-24 21:45:00,073XX S JEFFERY BLVD,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,APARTMENT,False,False,333,3.0,7.0,43.0,02,1190812.0,1856743.0,2023,2023-09-16 15:41:56,41.761919,-87.576209,"(41.7619185, -87.576209245)",32.0,22538.0,39.0,419.0,37.0,24.0,18.0,89.0,All Other Crime Types,2023-07,63.70,69.60


#### Grocery Store Data
This data apparently only applies to 2013 so I'm not sure if it will be useful for our overall model. If we wanted to do a snapshot analysis of 2013, we still have time series data that could be frequent enough for analysis. We can discuss this more.

In [20]:
grocery_stores = pd.read_csv('Grocery_Stores_-_2013_20231109.csv')
#Information about dataset here: https://github.com/Chicago/food-deserts

We could identify which areas are food deserts and create a 'Y'/'N' column for this using distance between the crime location and a grocery store. Food deserts are defined as areas in Chicago which are more than 0.5 or 1 mile from a grocery store, depending on the grocery store size. This is represented by the 'A' or 'B' buffer size. We could also represent this as the count of grocery stores within 0.5 or 1 mile.

We could also simply provide the count of grocery stores for each community area, but we would probably need to control for population size using census data that can give us population numbers by Community Area. I think we could try this first, see if any correlation exists, and if one does, we can investigate further and focus in on food deserts.

In [21]:
grocery_stores.columns.values

array(['STORE NAME', 'LICENSE ID', 'ACCOUNT NUMBER', 'SQUARE FEET',
       'BUFFER SIZE', 'ADDRESS', 'ZIP CODE', 'COMMUNITY AREA NAME',
       'COMMUNITY AREA', 'WARD', 'CENSUS TRACT', 'CENSUS BLOCK',
       'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE',
       'LOCATION'], dtype=object)

In [22]:
len(grocery_stores['COMMUNITY AREA'].unique())

75

In [23]:
grocery_store_count = grocery_stores.groupby(['COMMUNITY AREA']).agg(Grocery_Store_Count=('STORE NAME','count')).reset_index()

In [24]:
merged_crime_df = pd.merge(crime_and_police, grocery_store_count, how='left',left_on='Community Area',right_on='COMMUNITY AREA')

In [25]:
merged_crime_df.drop(columns='COMMUNITY AREA',inplace=True)

In [26]:
merged_crime_df.rename(columns={'Grocery_Store_Count':'Grocery Stores per Community Area'},inplace=True)

#### Housing Data
This data is clean enough and we can provide the number of affordable housing units per community area, but since there is no time column, I'm unsure how we could use this to train our model. This data was updated in October 2023, but there is no easy way to tell when each apartment/unit was built, so these numbers would likely only be reliable for the past 5-10 years. 

The grocery store data presents the same problem. We may need to think about separate analyses for time series vs. regression since some of our independent variables are so time restricted.

In [27]:
housing_df = pd.read_csv('Affordable_Rental_Housing_Developments_20231109.csv')

In [28]:
housing_df.rename(columns={'Community Area Number':'Community Area'}, inplace=True)

In [29]:
housing_df.head()

,Community Area Name,Community Area,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Near West Side,28,ARO,The Dylan,160 N. Morgan St.,60607,872.328.0028,Sterling Bay,28,1169706.814,1901381.768,41.884895,-87.652265,"(41.884895408116236, -87.65226488412628)"
1,Hegewisch,55,Senior,Senior Suites of Hegewisch,13550 S. Avenue O,60633,773-646-1333,Senior Lifestyle Corp.,116,1201038.059,1815487.733,41.648457,-87.540123,"(41.648457411436, -87.5401231660406)"
2,Uptown,3,Senior,Wilson Yard Senior Apts.,1032 W. Montrose Ave,60613,866-227-0438,Holsten Real Estate Dev. Corp.,13,1168438.959,1929421.650,41.961866,-87.656107,"(41.9618658532981, -87.656107428372)"
3,Lake View,6,ARO,The Mode,3462 N. Lincoln Ave.,60657,773-205-8300,606 Realty,2,1164033.827,1923207.502,41.944908,-87.672479,"(41.9449083304908, -87.6724793376448)"
4,North Lawndale,29,Multifamily,Lawndale Restoration II Apts. (scattered sites),10 N. Hamlin Ave.,60613,312-337-5339,Holsten Management Corp.,170,1151024.342,1899855.056,41.881092,-87.720910,"(41.8810923305496, -87.7209100899284)"


In [30]:
affordable_housing_units = housing_df.groupby('Community Area')['Units'].sum().reset_index()

In [31]:
affordable_housing_units.rename(columns={'Units':'Affordable Housing Units per Community Area',
                                        'Police Safety Score':'Police Safety Score per Community Area',
                                        'Police Trust Score': 'Police Trust Score per Community Area'}
                                        ,inplace=True)

In [32]:
merged_crime_df = pd.merge(merged_crime_df,affordable_housing_units, how='left',on='Community Area')

In [33]:
merged_crime_df.head(0)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats,Primary Type Grouping,Year_Month,Police Safety Score,Police Trust Score,Grocery Stores per Community Area,Affordable Housing Units per Community Area


#### Census Data

In [34]:
census_df = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012_20231109.csv')

In [35]:
census_df.shape

(78, 9)

In [36]:
len(census_df['Community Area Number'].unique())

78

In [37]:
census_df.drop(columns='COMMUNITY AREA NAME',inplace=True)

In [38]:
census_df.columns = census_df.columns.str.lower() + " per Community Area"
census_df.columns  = census_df.columns.str.title()
census_df.rename(columns={'Community Area Number Per Community Area': 'Community Area'},inplace=True)

In [39]:
merged_crime_df = pd.merge(merged_crime_df, census_df, how='left', on='Community Area')

### Train Station Data

In [57]:
train_station_df = pd.read_csv('train_stops_converted.csv')

In [59]:
train_station_df.columns.values

array(['Year', 'Month', 'Station_ID', 'Station_Name', 'Avg_Weekday_Rides',
       'Avg_Saturday_Rides', 'Avg_Sunday/Holiday_Rides', 'Monthly_Total',
       'Lat_Lon', 'Community', 'Comm_Num', 'SHAPE_AREA', 'SHAPE_LEN'],
      dtype=object)

In [60]:
train_station_df['Month'] = pd.to_datetime(train_station_df['Month'])

In [61]:
train_station_df['Year_Month'] = train_station_df['Month'].dt.to_period('M')

In [100]:
average_train_stats = train_station_df.groupby(['Comm_Num','Year_Month'])['Avg_Weekday_Rides','Avg_Saturday_Rides',
                        'Avg_Sunday/Holiday_Rides','Monthly_Total'].mean().reset_index()

/var/folders/1s/stcy42l14bd01cnvp2y2xhzr0000gn/T/ipykernel_5935/1332910167.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_train_stats = train_station_df.groupby(['Comm_Num','Year_Month'])['Avg_Weekday_Rides','Avg_Saturday_Rides',


In [102]:
average_train_stats.rename(columns={'Comm_Num':'Community Area'},inplace=True)

In [107]:
len(average_train_stats['Community Area'].unique())

42

In [104]:
merged_crime_df = pd.merge(merged_crime_df, average_train_stats, how='left', on=['Community Area','Year_Month'])

In [40]:
#merged_crime_df.to_csv('merged_crime_df.csv',index=False)

In [116]:
merged_crime_df.sort_values(by='Community Area')

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats,Primary Type Grouping,Year_Month,Police Safety Score,Police Trust Score,Grocery Stores per Community Area,Affordable Housing Units per Community Area,Percent Of Housing Crowded Per Community Area,Percent Households Below Poverty Per Community Area,Percent Aged 16+ Unemployed Per Community Area,Percent Aged 25+ Without High School Diploma Per Community Area,Percent Aged Under 18 Or Over 64 Per Community Area,Per Capita Income Per Community Area,Hardship Index Per Community Area,Avg_Weekday_Rides,Avg_Saturday_Rides,Avg_Sunday/Holiday_Rides,Monthly_Total
2634516,4892486,HM507556,2006-07-29 18:00:00,050XX N CUMBERLAND AVE,0820,THEFT,$500 AND UNDER,ALLEY,False,False,1614,16.0,41.0,0.0,06,1119317.0,1932237.0,2006,2018-02-28 15:56:25,41.970515,-87.836651,"(41.970514812, -87.83665128)",34.0,4453.0,75.0,181.0,29.0,31.0,12.0,44.0,All Other Crime Types,2006-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1508768,3134891,HK123523,2004-01-13 19:06:57,051XX S RUTHERFORD AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,811,8.0,23.0,0.0,08B,1132250.0,1869819.0,2004,2021-09-07 15:41:02,41.799015,-87.790546,"(41.799015491, -87.790546423)",35.0,22268.0,53.0,588.0,6.0,7.0,13.0,131.0,All Other Crime Types,2004-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1345495,2959051,HJ628376,2003-09-14 00:45:00,046XX W 31ST ST OVERPASS,1330,CRIMINAL TRESPASS,TO LAND,OTHER RAILROAD PROP / TRAIN DEPOT,False,False,1031,10.0,22.0,0.0,26,NaN,NaN,2003,2015-08-17 15:03:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,All Other Crime Types,2003-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1152510,2540072,HJ119804,2003-01-11 08:55:00,050XX N CUMBERLAND AVE,0560,ASSAULT,SIMPLE,STREET,False,False,1614,16.0,41.0,0.0,08A,1119317.0,1932301.0,2003,2018-02-28 15:56:25,41.970690,-87.836650,"(41.970690436, -87.836649916)",34.0,4453.0,75.0,181.0,29.0,31.0,12.0,44.0,All Other Crime Types,2003-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1131257,2655567,HJ266893,2003-03-28 23:50:00,051XX S RUTHERFORD AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,811,8.0,23.0,0.0,14,1132252.0,1869738.0,2003,2018-02-10 15:50:01,41.798793,-87.790541,"(41.798793178, -87.790540964)",35.0,22268.0,53.0,588.0,6.0,7.0,13.0,131.0,All Other Crime Types,2003-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645986,4783166,HM396729,2006-05-30 13:00:00,032XX W MONROE ST,0890,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",False,False,1124,11.0,28.0,NaN,06,1154551.0,1899349.0,2006,2018-02-28 15:56:25,41.879634,-87.707974,"(41.879633871, -87.707973888)",11.0,21572.0,28.0,765.0,23.0,30.0,16.0,123.0,All Other Crime Types,2006-05,NaN,NaN,NaN,NaN,4.7,19.7,12.9,19.5,33.5,28202.0,NaN,NaN,NaN,NaN,NaN
2668618,4785386,HM396793,2006-06-06 10:45:49,032XX W MONROE ST,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,1124,11.0,28.0,NaN,08B,1154551.0,1899349.0,2006,2018-02-28 15:56:25,41.879634,-87.707974,"(41.879633871, -87.707973888)",11.0,21572.0,28.0,765.0,23.0,30.0,16.0,123.0,All Other Crime Types,2006-06,NaN,NaN,NaN,NaN,4.7,19.7,12.9,19.5,33.5,28202.0,NaN,NaN,NaN,NaN,NaN
2685262,4791311,HM401461,2006-06-08 12:50:00,032XX W MONROE ST,0460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,False,1124,11.0,28.0,NaN,08B,1154551.0,1899349.0,2006,2018-02-28 15:56:25,41.879634,-87.707974,"(41.879633871, -87.707973888)",11.0,21572.0,28.0,765.0,23.0,30.0,16.0,123.0,All Other Crime Types,20

In [ ]:
merged_crime_df.melt()

In [113]:
merged_crime_df.columns.values

array(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic',
       'Beat', 'District', 'Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards',
       'Boundaries - ZIP Codes', 'Police Districts', 'Police Beats',
       'Primary Type Grouping', 'Year_Month', 'Police Safety Score',
       'Police Trust Score', 'Grocery Stores per Community Area',
       'Affordable Housing Units per Community Area',
       'Percent Of Housing Crowded Per Community Area',
       'Percent Households Below Poverty Per Community Area',
       'Percent Aged 16+ Unemployed Per Community Area',
       'Percent Aged 25+ Without High School Diploma Per Community Area',
       'Percent Aged Under 18 Or Over 64 Per Community Area',
       'Per Capita Income  Pe

In [118]:
merged_crime_df.groupby(['Community Area']).agg({'ID':'count','Police Safety Score':'mean','Police Trust Score':'mean', 
       'Grocery Stores per Community Area':'mean',
       'Affordable Housing Units per Community Area':'mean',
       'Percent Of Housing Crowded Per Community Area':'mean',
       'Percent Households Below Poverty Per Community Area':'mean',
       'Percent Aged 16+ Unemployed Per Community Area':'mean',
       'Percent Aged 25+ Without High School Diploma Per Community Area':'mean',
       'Percent Aged Under 18 Or Over 64 Per Community Area':'mean',
       'Per Capita Income  Per Community Area':'mean',
       'Hardship Index Per Community Area':'mean', 'Avg_Weekday_Rides':'mean',
       'Avg_Saturday_Rides':'mean', 'Avg_Sunday/Holiday_Rides':'mean', 'Monthly_Total':'mean'})

,ID,Police Safety Score,Police Trust Score,Grocery Stores per Community Area,Affordable Housing Units per Community Area,Percent Of Housing Crowded Per Community Area,Percent Households Below Poverty Per Community Area,Percent Aged 16+ Unemployed Per Community Area,Percent Aged 25+ Without High School Diploma Per Community Area,Percent Aged Under 18 Or Over 64 Per Community Area,Per Capita Income Per Community Area,Hardship Index Per Community Area,Avg_Weekday_Rides,Avg_Saturday_Rides,Avg_Sunday/Holiday_Rides,Monthly_Total
Community Area,,,,,,,,,,,,,,,,
0.0,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,34879,55.405038,54.824962,13.0,1178.0,7.7,23.6,8.7,18.2,27.5,23939.0,39.0,4011.726954,2909.598996,2056.078154,108018.580423
2.0,25995,55.497132,55.266520,18.0,324.0,7.8,17.2,8.8,20.8,38.5,23040.0,46.0,NaN,NaN,NaN,NaN
3.0,34073,59.384158,60.293151,15.0,1410.0,3.8,24.0,8.9,11.8,22.2,35787.0,20.0,3487.755445,2396.820678,1708.478664,92950.856214
4.0,15320,57.831152,58.541257,4.0,36.0,3.4,10.9,8.2,13.4,25.5,37524.0,17.0,2101.350127,1226.834930,753.340603,53708.274646
5.0,14075,60.473778,61.649259,6.0,162.0,0.3,7.5,5.2,4.5,26.2,57123.0,6.0,2033.215962,992.516457,580.860147,50430.040980
6.0,38692,60.385796,61.568439,9.0,510.0,1.1,11.4,4.7,2.6,17.0,60058.0,5.0,4731.459936,3518.049231,2504.735975,128269.517591
7.0,33289,61.444297,62.139108,11.0,161.0,0.8,12.3,5.1,3.6,21.5,71551.0,2.0,6939.998355,4148.495386,2781.827830,179199.114327
8.0,70366,61.915829,62.304597,17.0,1241.0,1.9,12.9,7.0,2.5,22.6,88669.0,1.0,6609.089225,4836.661507,3323.839265,177926.269924
